In [7]:
import numpy as np
from osgeo import gdal, ogr
from sklearn import metrics
import pandas as pd

naip_fn = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/channelFusion_Wetzwil_test.tif"

driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)

test_fn = "D:/cbr/VP2/VP2_SoilMapping/Shapes/test.shp"
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)

truth = target_ds.GetRasterBand(1).ReadAsArray()
pred_ds = gdal.Open('D:/cbr/VP2/VP2_SoilMapping/Processed/classified_Wetzwil_mitDEM.tif')
pred = pred_ds.GetRasterBand(1).ReadAsArray()

idx = np.nonzero(truth)
cm = metrics.confusion_matrix(truth[idx], pred[idx])

print(cm)

corr_matrix = pd.DataFrame(data=cm, index=["Grey Cluster", "Other"], columns=["Pred. Grey Cluster", "Pred. Other"])
print(corr_matrix)

print(" ")
print("######## Statistics ######### ")

accuracy = cm.diagonal() / cm.sum(axis=0)
print("Accuracy :                   ", accuracy)

A = cm[0][0]
B = cm[0][1]
C = cm[1][0]
D = cm[1][1]
P_corr = ((A + B) / (A + B + C + D)) * ((A + C) / (A + B + C + D))
P_incorr = ((C + D) / (A + B + C + D)) * ((B + D) / ( A + B + C + D))
Kappa = P_corr + P_incorr
print("Kappa Index :                ", Kappa)

FAR = (cm[0][1]) / ((cm[0][1]) + (cm[0][0]))
print("False Alarm Ratio :          ", FAR)

POD = (cm[0][0]) / ((cm[0][0]) + (cm[1][0]))
print("Propability of Detection :    ",POD)

[[20  0]
 [ 0  6]]
              Pred. Grey Cluster  Pred. Other
Grey Cluster                  20            0
Other                          0            6
 
######## Statistics ######### 
Accuracy :                    [1. 1.]
Kappa Index :                 0.6449704142011835
False Alarm Ratio :           0.0
Propability of Detection :     1.0
